In [1]:
import tkinter as tk
from rdflib import Graph, Namespace
import random

# 定义命名空间
gn = Namespace('http://www.geonames.org/ontology#')
geographis = Namespace('http://telegraphis.net/ontology/geography/geography#')

# 加载 RDF 文件
g = Graph()
g.parse('C:/Users/qq264/Desktop/countries.rdf', format='xml')

# 创建主窗口
root = tk.Tk()
root.title("Quiz Generator")

# 定义函数以生成测试题
def generate_quiz():
    # 编写 SPARQL 查询获取所有国家的名称和首都
    query = """
        PREFIX gn: <http://www.geonames.org/ontology#>
        PREFIX geographis: <http://telegraphis.net/ontology/geography/geography#>
        SELECT ?country ?name ?capital WHERE {
            ?country a gn:Country .
            ?country gn:name ?name .
            ?country geographis:capital ?capital .
        }
    """

    # 执行查询并生成测试题
    res = g.query(query)
    rows = list(res)

    row = random.choice(rows)
    country_uri, country_name, capital_uri = row
    country = str(country_name)
    if capital_uri.startswith("http"):
        capital = capital_uri.split('#')[-1]
    else:
        capital = str(capital_uri)
    options = [capital]

    # 从 RDF 数据中随机选择三个不同的国家/首都作为选项
    choices_query = f"""
        SELECT ?country ?capital WHERE {{
            ?country a gn:Country .
            ?country geographis:capital ?capital .
            FILTER (?country != <{country_uri}>)
        }}
        ORDER BY RAND() LIMIT 3
    """

    choices_res = g.query(choices_query)

    for choice in choices_res:
        _, incorrect_capital_uri = choice
        if incorrect_capital_uri.startswith("http"):
            incorrect_capital = incorrect_capital_uri.split('#')[-1]
        else:
            incorrect_capital = str(incorrect_capital_uri)
        options.append(incorrect_capital)

    # 将选项随机排列并输出测试题
    random.shuffle(options)
    question = f"Select the capital of {country}:"
    for i, option in enumerate(options):
        question += f"\n{chr(65 + i)}. {option}"
    quiz_label.config(text=question)

# 创建生成测试按钮
generate_button = tk.Button(root, text="Generate Quiz", command=generate_quiz)
generate_button.pack(pady=10)

# 创建测试题标签
quiz_label = tk.Label(root, wraplength=400, justify='left')
quiz_label.pack()

# 运行主循环
root.mainloop()


In [10]:
from tkinter import *
import rdflib

root = Tk()

def generate():
    g = rdflib.Graph()
    g.parse("C:/Users/qq264/Desktop/countries.rdf")
    qres = g.query("""
        PREFIX gn: <http://telegraphis.net/ontology/geography/geography#>
        SELECT ?t ?def  
        WHERE {?a gn:capital ?t;   }
        ORDER BY RAND()
        LIMIT 5""")
    global item_arr

    item_arr = []
    for row in qres:
        items = "What country's capital is ' "+str(row[0].split("#")[-1])+"'"
        item_arr.append(items)

    for item in range(len(item_arr)):
        str_number = str(item+1)
        the_item = str_number+". "+item_arr[item]
        global label
        label = Label(frame, text=the_item, width="50", height="2",
                      bg="#f2f2f2", fg="#333", font=("Helvetica", 11),
                      justify=LEFT, anchor=W)
        label.pack(pady=7)

    global answer_box
    answer_box = Entry(frame, width=40, font=("Helvetica", 12))
    answer_box.pack(pady=10)

    global submit_button
    submit_button = Button(frame, text="Submit", fg="white",
                            bg="#333", font=("Helvetica", 12),
                            command=save_answers)
    submit_button.pack(pady=10, ipadx=10, ipady=5)

def save_answers():
    with open("answers.txt", "a") as f:
        for i, item in enumerate(item_arr):
            f.write(f"{i+1}. {item}\n")
        f.write(f"Answer: {answer_box.get()}\n\n")
        answer_box.delete(0, END)

root.geometry("600x500+300+400")
root.title("Test Generator")

frame = Frame(root, bg="#fff")
frame.pack(expand=True, padx=20, pady=20)

generate_button = Button(frame, text="Generate", fg="white",
                         bg="#333", font=("Helvetica", 12),
                         command=generate)
generate_button.pack(pady=20, ipadx=10, ipady=5)

root.mainloop()
